# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS for 2 bedroom

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc
from AMI_BANDS_2021 import *

In [3]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',
             '0400118','0400119','0400120','0400121','0400122','0400123',
             '0400125','0400128','0400112','0400129']

#variables needed: household size NP, income INCP, & income adjustment factor
data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,TEN'

#assuming July 2021 $1 for HHInc equivalent to estimate 2023 Inc equivalent
inflation_adjust = 1.11

In [4]:
#if there's time, it would be better to upload a dict of prices from MAG
#then generate the dictionary through calcs in python (vs. excel)
#flag for later

#Income to afford median household rent price + utilities by PUMA
#3 bedroom multifamily unit
inc_needed_mf = {'0400112':108100,'0400113':82800,'0400114':68770 ,'0400115':64285,
              '0400116':63250,'0400117':75900,'0400118':76475,'0400119':56925,
              '0400120':74750,'0400121':58650,'0400122':54970,'0400123':54970,
              '0400125':55154,'0400128':66700,'0400129':79350}

 
#Income to afford median household sales price by PUMA
#3 bedroom single-family unit
inc_needed_sf = {'0400112':110308,'0400113':83214,'0400114':67390,'0400115':63894,
              '0400116':56856,'0400117':65918,'0400118':70334,'0400119':50876,
              '0400120':79350,'0400121':62238,'0400122':52164,'0400123':68540,
              '0400125':55706,'0400128':72634,'0400129':80960}

In [5]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [30]:
df = get.get_puma(sample,y1,data_cols)

In [31]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [32]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)

In [33]:
df['inc_needed_sf'] = df['GEO_ID'].map(inc_needed_sf)
df['inc_needed_mf'] = df['GEO_ID'].map(inc_needed_mf)

In [36]:
dff = df[~(df.HHSz.isna())&(df.HINCP!=-60000)&(df.HINCP!=0)&(df.TEN==3.0)].copy()
dff['HINCP'] = dff.ADJINC * dff.HINCP

In [38]:
dff['can_rent_sf'] = np.where(dff.HINCP>=dff.inc_needed_sf,'can rent','cannot afford')
dff['can_rent_mf'] = np.where(dff.HINCP>=dff.inc_needed_mf,'can rent','cannot afford')

In [39]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),inc_lbl[0],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),inc_lbl[1],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),
                   inc_lbl[2],np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),
                   inc_lbl[3],np.where((dff['HINCP']>dff['HHSz'].map(AMI_100pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_120pct)),
                                       inc_lbl[4],inc_lbl[5])))))

In [40]:
#make a column for each PUMA that assesses the number of households 
sf_cols=[]
for k in inc_needed_sf.keys():
    dff[f'aff_{k}_can_sf']=np.where(dff.HINCP>=inc_needed_sf[k],1,0)
    dff[f'aff_{k}_cant_sf']=np.where(dff.HINCP<inc_needed_sf[k],1,0)
    dff[f'aff_{k}_can_sf_wt']=dff[f'aff_{k}_can_sf']*dff.WGTP
    dff[f'aff_{k}_cant_sf_wt']=dff[f'aff_{k}_cant_sf']*dff.WGTP
    sf_cols = sf_cols + [f'aff_{k}_can_sf',f'aff_{k}_cant_sf',
                        f'aff_{k}_can_sf_wt',f'aff_{k}_cant_sf_wt']
    #sf_cols.append(f'aff_{k}_can_sf')
    #sf_cols.append(f'aff_{k}_cant_sf')
    #sf_cols.append(f'aff_{k}_can_sf_wt')
    #sf_cols.append(f'aff_{k}_cant_sf_wt')

In [41]:
#make a column for each PUMA that assesses the number of households 
mf_cols=[]
for k in inc_needed_mf.keys():
    dff[f'aff_{k}_can_mf']=np.where(dff.HINCP>=inc_needed_mf[k],1,0)
    dff[f'aff_{k}_cant_mf']=np.where(dff.HINCP<inc_needed_mf[k],1,0)
    dff[f'aff_{k}_can_mf_wt']=dff[f'aff_{k}_can_mf']*dff.WGTP
    dff[f'aff_{k}_cant_mf_wt']=dff[f'aff_{k}_cant_mf']*dff.WGTP
    mf_cols = mf_cols+[f'aff_{k}_can_mf',f'aff_{k}_cant_mf',
                       f'aff_{k}_can_mf_wt',f'aff_{k}_cant_mf_wt']

/var/folders/s1/ytnwss3951g16cgxh_xr_rq00000gp/T/ipykernel_92254/3225957969.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff[f'aff_{k}_cant_mf']=np.where(dff.HINCP<inc_needed_mf[k],1,0)
/var/folders/s1/ytnwss3951g16cgxh_xr_rq00000gp/T/ipykernel_92254/3225957969.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dff[f'aff_{k}_can_mf_wt']=dff[f'aff_{k}_can_mf']*dff.WGTP
/var/folders/s1/ytnwss3951g16cgxh_xr_rq00000gp/T/ipykernel_92254/3225957969.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

In [42]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,TEN,WGTP1,WGTP2,WGTP3,WGTP4,...,aff_0400125_can_mf_wt,aff_0400125_cant_mf_wt,aff_0400128_can_mf,aff_0400128_cant_mf,aff_0400128_can_mf_wt,aff_0400128_cant_mf_wt,aff_0400129_can_mf,aff_0400129_cant_mf,aff_0400129_can_mf_wt,aff_0400129_cant_mf_wt
3612,0400120,177353.6016,3.0,42.0,1.029928,3.0,74.0,13.0,71.0,44.0,...,42.0,0.0,1,0,42.0,0.0,1,0,42.0,0.0
3637,0400119,55307.1336,7.0,172.0,1.029928,3.0,70.0,81.0,157.0,246.0,...,172.0,0.0,0,1,0.0,172.0,0,1,0.0,172.0
3639,0400116,70344.0824,2.0,177.0,1.029928,3.0,244.0,245.0,293.0,314.0,...,177.0,0.0,1,0,177.0,0.0,0,1,0.0,177.0


### table by PUMA for renters by AMI range - can afford/can't afford median rent

In [43]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [44]:
drop_cols = ['HINCP','NP','ADJINC','HHSz','TEN','inc_needed_sf','inc_needed_mf']

In [45]:
table = dff.copy().drop(columns=drop_cols)

In [46]:
table.head()

,GEO_ID,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,WGTP7,WGTP8,...,aff_0400125_can_mf_wt,aff_0400125_cant_mf_wt,aff_0400128_can_mf,aff_0400128_cant_mf,aff_0400128_can_mf_wt,aff_0400128_cant_mf_wt,aff_0400129_can_mf,aff_0400129_cant_mf,aff_0400129_can_mf_wt,aff_0400129_cant_mf_wt
3612,0400120,42.0,74.0,13.0,71.0,44.0,13.0,12.0,13.0,40.0,...,42.0,0.0,1,0,42.0,0.0,1,0,42.0,0.0
3637,0400119,172.0,70.0,81.0,157.0,246.0,48.0,270.0,136.0,197.0,...,172.0,0.0,0,1,0.0,172.0,0,1,0.0,172.0
3639,0400116,177.0,244.0,245.0,293.0,314.0,152.0,182.0,274.0,266.0,...,177.0,0.0,1,0,177.0,0.0,0,1,0.0,177.0
3642,0400128,114.0,98.0,126.0,149.0,33.0,101.0,187.0,88.0,33.0,...,0.0,114.0,0,1,0.0,114.0,0,1,0.0,114.0
3645,0400120,73.0,73.0,68.0,119.0,77.0,126.0,21.0,23.0,68.0,...,0.0,73.0,0,1,0.0,73.0,0,1,0.0,73.0


In [47]:
rent_ami.head()

,GEO_ID,AMI_range,can_rent_sf,can_rent_mf,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80


In [48]:
rent_ami = table.copy().drop(columns=sf_cols+mf_cols).groupby(['GEO_ID','AMI_range','can_rent_sf','can_rent_mf']).sum().reset_index()
rent_ami = make_est(rent_ami)
rent_ami = rent_ami.drop(columns=repwts)
rent_ami = pd.pivot_table(rent_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',
                          columns=['AMI_range','can_rent_sf','can_rent_mf'],aggfunc=np.sum).reset_index()

In [49]:
rent_tot = table.copy().drop(columns=sf_cols+mf_cols+['AMI_range']).groupby(['GEO_ID','can_rent_sf','can_rent_mf']).sum().reset_index()
rent_tot = make_est(rent_tot)
rent_tot = rent_tot.drop(columns=repwts)
rent_tot = pd.pivot_table(rent_tot,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',
                          columns=['can_rent_sf','can_rent_mf'],aggfunc=np.sum).reset_index()

In [50]:
table_sf = table.copy().drop(columns=['AMI_range','can_rent_sf','can_rent_mf']+mf_cols).groupby(['GEO_ID']).sum().reset_index()
table_sf = make_est(table_sf)
table_sf = table_sf.drop(columns=repwts)

In [51]:
table_mf = table.copy().drop(columns=['AMI_range','can_rent_sf','can_rent_mf']+sf_cols).groupby(['GEO_ID']).sum().reset_index()
table_mf = make_est(table_mf)
table_mf = table_mf.drop(columns=repwts)

In [52]:
table_sf.head()

,GEO_ID,hh,aff_0400112_can_sf,aff_0400112_cant_sf,aff_0400112_can_sf_wt,aff_0400112_cant_sf_wt,aff_0400113_can_sf,aff_0400113_cant_sf,aff_0400113_can_sf_wt,aff_0400113_cant_sf_wt,...,aff_0400128_cant_sf,aff_0400128_can_sf_wt,aff_0400128_cant_sf_wt,aff_0400129_can_sf,aff_0400129_cant_sf,aff_0400129_can_sf_wt,aff_0400129_cant_sf_wt,hh_SE,hh_MOE,hh_CV
0,0400112,11304.0,26,49,3991.0,7313.0,36,39,6093.0,5211.0,...,34,6641.0,4663.0,37,38,6227.0,5077.0,1031.845083,1697.385161,5.549022
1,0400113,12204.0,23,78,3083.0,9121.0,37,64,4532.0,7672.0,...,56,5849.0,6355.0,38,63,4584.0,7620.0,807.449271,1328.254051,4.022047
2,0400114,18388.0,25,129,2950.0,15438.0,41,113,4991.0,13397.0,...,107,5910.0,12478.0,44,110,5456.0,12932.0,993.351411,1634.063070,3.283995
3,0400115,19263.0,17,130,1709.0,17554.0,28,119,2922.0,16341.0,...,111,3526.0,15737.0,30,117,3106.0,16157.0,1108.569991,1823.597635,3.498431
4,0400116,27680.0,22,248,2205.0,25475.0,58,212,5459.0,22221.0,...,205,6426.0,21254.0,59,211,5573.0,22107.0,1227.622367,2019.438793,2.696080


In [53]:
with pd.ExcelWriter(f'../../output/pums_gap_rent/pums_rent_gap_marketdata_120AMI_2BR_renterOnly.xlsx') as writer:
    rent_ami.to_excel(writer,sheet_name='can_cannot_withinpuma_ami.xlsx')
    rent_tot.to_excel(writer,sheet_name='can_cannot_withinpuma.xlsx')
    table_sf.to_excel(writer,sheet_name='can_cannot_all_sf.xlsx')
    table_mf.to_excel(writer,sheet_name='can_cannot_all_mf.xlsx')